In [1]:
import pandas as pd
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split

In [2]:
class BPR(object):
    def __init__(self):
        self.W = None             # user matrix
        self.H = None             # item matrix
        
        self.Wsc = None        # scorer
        self.Hsc = None 
        self.rating_exp = {}   # softmax sum
        self.rating_exp_mul_H = {}
        
        self.uid = None            # uid,iid without duplicates
        self.iid = None
        
        self.user_items = {}       # 用户u对应他访问过的所有items集合
        self.dev_user_items = {}
        
        self.uid_dict = None      # serialize uid and iid
        self.iid_dict = None      #  {(original id in dataset): (serial_idx)}
        self.uid_dict_rev = None  # reverse key and value
        self.iid_dict_rev = None  #  {(serial_idx): (original id in dataset)}

    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio, shuffle = True, random_state = 5)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test
    
    def split(self, df, train_size=0.8, test_size = 0.1):
        self.uid = np.asarray(list(set(df.iloc[:,0].values)))
        self.iid = np.asarray(list(set(df.iloc[:,1].values)))
        self.uid.sort()
        self.iid.sort()
        self.uid_dict = dict(zip(self.uid, [i for i in range(len(self.uid))]))
        self.iid_dict = dict(zip(self.iid, [i for i in range(len(self.iid))]))
        self.uid_dict_rev = {v : k for k, v in self.uid_dict.items()}
        self.iid_dict_rev = {v : k for k, v in self.iid_dict.items()}
        train, test = self._split(df, train_size)
        test, dev = self._split(test, test_size / (1 - train_size))
        return train, test, dev
    
    def generate_train_batch(self, batch, sets):
        train = []
        for b in range(batch):
            u = self.uid[random.randint(0, self.uid.size - 1)]
            i = sets[u][random.randint(0, sets[u].size - 1)]
            j = self.iid[random.randint(0, self.iid.size - 1)]
            while j in sets[u]:
                j = self.iid[random.randint(0, self.iid.size - 1)]
            train.append([self.uid_dict[u], self.iid_dict[i], self.iid_dict[j]])
        return np.asarray(train)   
    
    def scorer_prob(self, uid, iid):              # Softmax probability, uids, iids serial
        if uid not in self.rating_exp.keys():
            r = 0
            h = 0
            for i in self.user_items[self.uid_dict_rev[uid]]:
                temp = np.exp(np.dot(self.Wsc[uid], self.Hsc[self.iid_dict[i]]))
                r += temp
                h += temp * self.Hsc[self.iid_dict[i]]
            self.rating_exp[uid] = r
            self.rating_exp_mul_H[uid] = h
        
        return np.exp(np.dot(self.Wsc[uid], self.Hsc[iid])) / self.rating_exp[uid]
    
    def fit_dds(self, df, dev, k, stepsize=0.05, regulation_rate=0.0001, max_iter=500, batch=10000):
        self.W = np.random.rand(len(self.uid), k)*0.01      # 初始化 W，H
        self.H = np.random.rand(len(self.iid), k)*0.01
        self.Wsc = np.random.rand(len(self.uid), k)*0.01      # 初始化 scorer
        self.Hsc = np.random.rand(len(self.iid), k)*0.01
            
        for u in self.uid:                                # 创建字典：用户u对应他访问过的所有items集合
            self.user_items[u] = df[df.iloc[:, 0]==u].iloc[:, 1].values
            self.dev_user_items[u] = dev[dev.iloc[:, 0]==u].iloc[:, 1].values     
                            
        for x in range(max_iter):             # Use stochastic gradient descent method to solve W & H
            loss = 0
            W_grad = np.zeros((len(self.uid), k))
            H_grad = np.zeros((len(self.iid), k))
            
            uij = self.generate_train_batch(batch, self.user_items)
            for u, i, j in uij:
                
                scorer = self.scorer_prob(u, i)
                
                xuij = np.dot(self.W[u], self.H[i]) - np.dot(self.W[u], self.H[j])
                sigmoid = 1.0 / (1 + math.exp(xuij))
                loss += -np.log(1.0/(1 + math.exp(-xuij)))
                
                # DDS
                
                W_grad[u] += scorer  * (sigmoid * (self.H[i] - self.H[j]) + regulation_rate * self.W[u])
                H_grad[i] += scorer * (sigmoid * self.W[u] + regulation_rate * self.H[i])
                H_grad[j] += scorer  * (-sigmoid * self.W[u] + regulation_rate * self.H[j])
   
            self.W += W_grad
            self.H += H_grad
            
            W_grad_dev = np.zeros((len(self.uid), k))
            H_grad_dev = np.zeros((len(self.iid), k))
                
            for u, i, j in self.generate_train_batch(5000, self.dev_user_items):
                xuij = np.dot(self.W[u], self.H[i]) - np.dot(self.W[u], self.H[j])
                sigmoid = 1.0 / (1 + math.exp(xuij))
                W_grad_dev[u] += stepsize * (sigmoid * (self.H[i] - self.H[j]) + regulation_rate * self.W[u])
                H_grad_dev[i] += stepsize * (sigmoid * self.W[u] + regulation_rate * self.H[i])
                H_grad_dev[j] += stepsize * (-sigmoid * self.W[u] + regulation_rate * self.H[j])
                
            r_W = np.sum(W_grad * W_grad_dev, axis = 1)
            r_H = np.sum(H_grad * H_grad_dev, axis = 1)
            r_W = np.expand_dims(r_W, axis=0).repeat(k, axis = 0).T
            r_H = np.expand_dims(r_H, axis=0).repeat(k, axis = 0).T

            log_prob_W_grad = np.zeros((len(self.uid), k))
            log_prob_H_grad = np.zeros((len(self.iid), k))
            
            for b in range(batch):
                u = uij[b,0]
                i = uij[b,1]
                log_prob_W_grad[u] = self.Hsc[i] - self.rating_exp_mul_H[u] / self.rating_exp[u]
                log_prob_H_grad[i] = self.Wsc[u]*(1 - self.scorer_prob(u,i))
            self.Wsc += r_W * log_prob_W_grad
            self.Hsc += r_H * log_prob_H_grad

            self.rating_exp = {}
            self.rating_exp_mul_H = {}
            
            if x == max_iter - 1:
                print(f"Iteration: {x+1}, BPR loss: {loss / batch}")
                
    def fit_normal(self, df, k, stepsize=0.05, regulation_rate=0.0001, max_iter=500, batch=10000):
        self.W = np.random.rand(len(self.uid), k)*0.01      # 初始化 W，H
        self.H = np.random.rand(len(self.iid), k)*0.01
        
        for u in self.uid:                                # 创建字典：用户u对应他访问过的所有items集合
            self.user_items[u] = df[df.iloc[:, 0]==u].iloc[:, 1].values
                            
        for x in range(max_iter):             # Use stochastic gradient descent method to solve W & H
            loss = 0
            for u, i, j in self.generate_train_batch(batch, self.user_items):
                xuij = np.dot(self.W[u], self.H[i]) - np.dot(self.W[u], self.H[j])
                sigmoid = 1.0 / (1 + math.exp(xuij))
                loss += -np.log(1.0/(1 + math.exp(-xuij)))
                self.W[u] += stepsize * (sigmoid * (self.H[i] - self.H[j]) + regulation_rate * self.W[u])
                self.H[i] += stepsize * (sigmoid * self.W[u] + regulation_rate * self.H[i])
                self.H[j] += stepsize * (-sigmoid * self.W[u] + regulation_rate * self.H[j])
            if x == max_iter - 1:
                print(f"Iteration: {x+1}, BPR loss: {loss / batch}")
    
    def _predict(self, uid, items, n):
        top_N = []
        
        for i in range(len(items)):
            user = self.uid_dict[uid]
            item = self.iid_dict[items[i]]
            top_N.append((items[i], np.dot(self.W[user], self.H[item])))
                
        return sorted(top_N, key=lambda s: s[1], reverse=True)[:n]
    
    def NDCG(self, uid, test, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
        test_user = test[test.iloc[:, 0] == uid]
        rating = self._predict(uid, test_user.iloc[:, 1].values, n)
        irating =sorted(test_user.iloc[:, 2].values, reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating): n = len(irating)  
        for i in range(n):
            r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
            dcg += 1.0*(2**r - 1)/math.log(i + 2, 2)
            idcg += 1.0*(2**irating[i] - 1)/math.log(i + 2, 2)
        return dcg/idcg
    
    def performance(self, test, n):      # Output recall@n, precision@n, NDCG@n
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        for i in self.uid:
            # Items that User i hasn't tried in training set
            unknown_items = np.setdiff1d(self.iid, self.user_items[i])
            # Items that User i actually tried in testing set
            known_items = test[test.iloc[:, 0]==i].iloc[:, 1].values
            
            #目标：预测 unknown items 中的top_N，若击中test中的items，则为有效预测
            ru = self._predict(i, unknown_items, n)
            for item ,pui in ru:
                if item in known_items:
                    hit += 1
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, test, n)  
            
        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [3]:
df1 = pd.read_csv("./ml-100k/u.data", sep="\t", names=['user id', 'item id', 'rating', 'timestamp'])
df2 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

### 100K

In [4]:
model1 = BPR()
model2 = BPR()
train1, test1, dev1 = model1.split(df1)
train2, test2, dev2 = model2.split(df1)
print(train1.shape)
print(test1.shape)
print(dev1.shape)

(79619, 4)
(9942, 4)
(10439, 4)


### 100K Pure BPR

In [5]:
model1.fit_normal(train1, k = 50)

Iteration: 500, BPR loss: 0.054694487128717176


In [6]:
n = 10
rec, pre, ndcg = model1.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.13626723223753975
Recall@10: 0.1292496479581573
NDCG@10: 0.8197595966855103


### 100K BPR + Data Selection

In [7]:
model2.fit_dds(train1, dev1, k = 50)

Iteration: 500, BPR loss: 0.110238534522965


In [8]:
n = 10
rec, pre, ndcg = model2.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.16023329798515376
Recall@10: 0.151981492657413
NDCG@10: 0.8316703966743907


### 1M

In [9]:
model3 = BPR()
model4 = BPR()
train3, test3, dev3 = model3.split(df2)
train4, test4, dev4 = model4.split(df2)
print(train3.shape)
print(test3.shape)
print(dev3.shape)

(797758, 4)
(99692, 4)
(102759, 4)


### 1M Pure BPR

In [12]:
model3.fit_normal(train3, k = 50)

Iteration: 500, BPR loss: 0.051887580848845184


In [14]:
n = 10
rec, pre, ndcg = model3.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.1303287380699894
Recall@10: 0.1236169784751559
NDCG@10: 0.8148307094569092


### 1M BPR + Data Selection

In [13]:
model4.fit_dds(train3, dev3, k = 50)

Iteration: 500, BPR loss: 0.11402055514776264


In [15]:
n = 10
rec, pre, ndcg = model4.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.1610816542948038
Recall@10: 0.15278615972641318
NDCG@10: 0.8307697091757171
